# Set Label

In [47]:
real = ['LJSpeech-1.1', 'data']
generated = ['generated_audio']

# List up Dataset

In [61]:
import os
os.getcwd()
os.chdir('/root/NetChallenge10')

PATH = os.path.join(os.getcwd(), 'dataset')


In [62]:
def search_wav(label, path):
    temp = []

    for file in os.listdir(path):
        file_path = os.path.join(path, file)

        if(os.path.isdir(file_path)):
            temp += search_wav(label, file_path)
        elif(os.path.splitext(file_path)[-1] == '.wav'):
            temp.append([label, file_path])
    
    return temp
        

In [63]:
# real_dataset_list
label = 'real'
real_dataset_list = []
for folder in real:
    real_dataset_list += search_wav(label, os.path.join(PATH, folder))

# fake_dataset_list
label = 'fake'
fake_dataset_list = []
for folder in generated:
    fake_dataset_list += search_wav(label, os.path.join(PATH, folder))

# Analyze Dataset

In [69]:
print('length of real:', len(real_dataset_list))
print('length of fake:', len(fake_dataset_list))

length of real: 24018
length of fake: 134266


# Generate Balanced Dataset

In [70]:
import random
balanced_fake_data_list = random.sample(fake_dataset_list, len(real_dataset_list))
print(len(balanced_fake_data_list))
print(balanced_fake_data_list[:10])

24018
[['fake', '/root/NetChallenge10/dataset/generated_audio/ljspeech_melgan_large/LJ010-0256_gen.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/gen_6358.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen_5099.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/ljspeech_multi_band_melgan/LJ014-0243_gen.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/ljspeech_waveglow/LJ039-0061.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/ljspeech_waveglow/LJ050-0136.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/ljspeech_melgan_large/LJ013-0136_gen.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/ljspeech_full_band_melgan/LJ030-0122_gen.wav'], ['fake', '/root/NetChallenge10/dataset/generated_audio/common_voices_prompts_from_conformer_fastspeech2_pwg_ljspeech/generated/gen

# save as json

In [71]:
# save all
import json

PATH = "/root/NetChallenge10"
base_json = {"data":[]}

for label, path in real_dataset_list+fake_dataset_list:
    data = {
        "label" : label,
        "wav" : path
    }

    base_json['data'].append(data)

# save preprocessed data
os.makedirs(os.path.join(PATH, 'data'), exist_ok=True)
with open(os.path.join(PATH, 'data', 'total_data.json'),'w') as j:
    json.dump(base_json,j,ensure_ascii=False, indent=4)


In [72]:
# save all
import json

PATH = "/root/NetChallenge10"
base_json = {"data":[]}

for label, path in real_dataset_list+balanced_fake_data_list:
    data = {
        "label" : label,
        "wav" : path
    }

    base_json['data'].append(data)

# save preprocessed data
os.makedirs(os.path.join(PATH, 'data'), exist_ok=True)
with open(os.path.join(PATH, 'data', 'balanced_total_data.json'),'w') as j:
    json.dump(base_json,j,ensure_ascii=False, indent=4)


# Split train and test

In [75]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(base_json['data'], train_size=0.8, test_size=0.2, random_state=123, shuffle=True)
print(train_data[0])
print(test_data[0])
print(len(train_data), len(test_data))


{'label': 'fake', 'wav': '/root/NetChallenge10/dataset/generated_audio/ljspeech_melgan_large/LJ040-0089_gen.wav'}
{'label': 'real', 'wav': '/root/NetChallenge10/dataset/LJSpeech-1.1/LJSpeech-1.1/wavs/LJ002-0220.wav'}
38428 9608


In [76]:
train_path = os.path.join(PATH, 'data', 'train_preprocessed_data.json')
print("Train 데이터셋 저장")
train_json = {'data' : train_data}
with open(train_path,'w') as j:
    json.dump(train_json,j,ensure_ascii=False, indent=4)

test_path = os.path.join(PATH, 'data', 'test_preprocessed_data.json')
print("Test 데이터셋 저장")
test_json = {'data' : test_data}
with open(test_path,'w') as j:
    json.dump(test_json,j,ensure_ascii=False, indent=4)


Train 데이터셋 저장
Test 데이터셋 저장
